### Import and Load

In [401]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 50)
pd.set_option('display.float_format', '{:.2f}'.format)

df = pd.read_csv('WA_Fn-UseC_-Telco-Customer-Churn.csv')
# df.head()

### Análise Exploratória

In [400]:
c_id = 20                        # id da coluna em análise
# c = df.columns[c_id]
c = 'Contract'
# print(df[c].value_counts())
# print(df[c].describe())
# print(type(df[c][0]))

### Histograma

In [399]:
# plt.hist(df[c], normed=False, bins=20, alpha=0.5)

### Gráfico de Barras

In [398]:
# plt.figure(figsize=(9,5))
# plt.bar(df[c].value_counts().index, df[c].value_counts(), width=0.5)

### Check Missing Values

In [12]:
# print(pd.isnull(df).sum())     # Count the Null Columns
# print(df[df["gender"].isnull()])    # Single Column Is Null
# print(df[df.isnull().any(axis=1)].head())    # All Null Columns

### Check Mismatched Values

In [373]:
# df['TotalCharges'].str.findall(r'\D')   # verifica se há algum caracter na string que não é numérico
# type(df.TotalCharges[0])             # verifica o tipo de um determinado campo olhando a primera linha. Neste exemplo campo se chama 'TotalCharges'
# print(df.tenure.str.isnumeric())     # check se é numérico se a coluna for string. não este o caso aqui. ainda tenho dúvidas com esta função
# df.select_dtypes(include=["float", 'int64'])   # verifica quais colunas são numéricas
# df.select_dtypes(include=np.number)   # verifica quais colunas são numéricas. equivalente a anterior
# df.tenure                        # neste exemplo campo = 'tenure'. nesta função olhe o tipo do dado no final no campo dtype: int64 ou float ou str

### Transform

In [402]:
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder

df['TotalCharges'][df['TotalCharges'] == ' '] = '0'     # verifica as linhas cujo 'TotalCharges' == ' ' e atribui '0'
# df[df['TotalCharges'] == '0']['TotalCharges']
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'])  # transforma a coluna de string para numérico

df['Churn'][df['Churn'] == 'No'] = 0     # atriui 0 onde era 'No'
df['Churn'][df['Churn'] == 'Yes'] = 1

col = 'Dependents'
df[col][df[col] == 'No'] = 0     # atriui 0 onde era 'No'
df[col][df[col] == 'Yes'] = 1

col = 'Partner'
df[col][df[col] == 'No'] = 0     # atriui 0 onde era 'No'
df[col][df[col] == 'Yes'] = 1 

col = 'PaperlessBilling'
df[col][df[col] == 'No'] = 0     # atriui 0 onde era 'No'
df[col][df[col] == 'Yes'] = 1  

le = preprocessing.LabelEncoder()
col = 'TechSupport'
le.fit(df[col])
df.at[:,col] = le.transform(df[col])
dict_TechSupport = {'No': 0, 'Yes': 2, 'No internet service': 1}

col = 'StreamingTV'
le.fit(df[col])
df.at[:,col] = le.transform(df[col])
dict_TechSupport = {'No': 0, 'Yes': 2, 'No internet service': 1}

col = 'StreamingMovies'
le.fit(df[col])
df.at[:,col] = le.transform(df[col])
dict_TechSupport = {'No': 0, 'Yes': 2, 'No internet service': 1}

col = 'Contract'
le.fit(df[col])
df.at[:,col] = le.transform(df[col])
dict_Contract = {'Month-to-month': 0, 'Two year': 2, 'One year': 1}

col = 'PaymentMethod'
le.fit(df[col])
df.at[:,col] = le.transform(df[col])
dict_PaymentMethod = {'Electronic check': 2, 'Mailed check': 3, 'Bank transfer (automatic)': 0, 'Credit card (automatic)': 1}

col = 'gender'
le.fit(df[col])
df.at[:,col] = le.transform(df[col])
dict_TechSupport = {'Male': 1, 'Female': 0}

col = 'InternetService'
le.fit(df[col])
df.at[:,col] = le.transform(df[col])
dict_PaymentMethod = {'Fiber optic': 1, 'DSL': 0, 'No': 2}

col = 'OnlineBackup'
le.fit(df[col])
df.at[:,col] = le.transform(df[col])
dict_TechSupport = {'No': 0, 'Yes': 2, 'No internet service': 1}

col = 'PhoneService'
df[col][df[col] == 'No'] = 0     # atriui 0 onde era 'No'
df[col][df[col] == 'Yes'] = 1  

### Feature Selection

In [403]:
# df.columns
# Index(['customerID', 'gender', 'SeniorCitizen', 'Partner', 'Dependents', 'tenure', 'PhoneService', 'MultipleLines', 'InternetService',
#       'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling',
#       'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'Churn']

features = ['TechSupport', 'Contract', 'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'tenure', 'SeniorCitizen', 'Partner', 'StreamingMovies', 'gender']
y = df['Churn']
y = y.astype('int')

### One Hot Encoding

In [404]:
OHE = False
if OHE:
    col = 'Contract'
    df_ohe = df[features]
    ohe = OneHotEncoder(sparse=False) # categorical_features = boolean...
    # for col in features_ohe:
    le.fit(df[col])
    col2 = col + '_e'
    df.at[:,col2] = le.transform(df[col])
    colunas = list(df.groupby([col, col2]).max().index.levels[0])         

    X_ohe = ohe.fit_transform(df[col2].values.reshape(-1, 1)) # It returns an numpy array                
    new = pd.DataFrame(X_ohe, columns=colunas, index=df.index)
    df_ohe = pd.merge(new, df_ohe, how='inner', left_index=True, right_index=True)
    # df = df_ohe.copy()
    df = df_ohe.drop(col, axis=1)
else:
    df = df[features]

### Modelagem

In [405]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
from sklearn.preprocessing import MinMaxScaler

X = df

In [406]:
norm = False
# clf = RandomForestClassifier(n_estimators=100, max_features=5, n_jobs = -1, random_state = 0)  # n_estimators=600, 
clf = GradientBoostingClassifier(n_estimators=100, learning_rate = 0.1, max_depth = 4, random_state = 0)
# clf = KNeighborsClassifier(n_neighbors = 4)
# clf = LogisticRegression(C=0.1)

# norm = True
# clf = MLPClassifier(hidden_layer_sizes = [100, 100], solver='adam',  activation = 'tanh', alpha=0.00001, random_state = 0)   
   
# Fit
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0)
if norm:
    scaler = MinMaxScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

clf.fit(X_train, y_train)

y_proba_lr = clf.predict_proba(X_test)
fpr_lr, tpr_lr, _ = roc_curve(y_test, y_proba_lr[:,1])
roc_auc_lr = auc(fpr_lr, tpr_lr)

y_predicted = clf.predict(X_test)
clr = classification_report(y_test, y_predicted, target_names = ['0', '1'])

In [407]:
print('score train:', clf.score(X_train, y_train))
print('score test:', clf.score(X_test, y_test))
print('AUC', roc_auc_lr)
print(confusion_matrix(y_test, y_predicted))
print(clr)

score train: 0.8451344187807649
score test: 0.7847813742191937
AUC 0.8270091551381591
[[1141  157]
 [ 222  241]]
              precision    recall  f1-score   support

           0       0.84      0.88      0.86      1298
           1       0.61      0.52      0.56       463

    accuracy                           0.78      1761
   macro avg       0.72      0.70      0.71      1761
weighted avg       0.78      0.78      0.78      1761

